In [23]:
import os
import re
import pickle
import numpy as np
from scipy.sparse import dok_matrix
from tqdm import tqdm

In [24]:
def file_to_matrix(file):
    matrix = None
    while True:
        line = file.readline()
        if "@nr_states" in line:
            n_states = int(file.readline())
            matrix = dok_matrix((n_states, n_states), dtype=np.float64)
        if "state " in line:
            last = file.tell()
            state = int(re.findall(r'\d+', line)[0])
            _ = file.readline() # Skip "action 0" line
            line = file.readline()
            while line and not "state" in line:
                to_state = int(re.findall(r'\d+', line)[0])
                value = float(re.findall(r': \d+\.\d+|: \d+', line)[0][2:])
                matrix[state, to_state] = value
                last = file.tell()
                line = file.readline()
            file.seek(last)
        if not line:
            break
    return matrix

In [25]:
DIR = './data/downloaded'
matrices = []

for filename in tqdm(os.listdir(DIR)):
    if not filename.lower().endswith('.drn'):
        continue
    f = os.path.join(DIR, filename)
    if os.path.isfile(f):
        with open(f, "r") as file:
            try:
                matrix = file_to_matrix(file)
            except MemoryError:
                print(f"{filename} takes too much memory and it was skipped")
                continue
            with open(f"./data/downloaded/{filename[:-4]}.pickle", 'wb') as save_file:
                pickle.dump(matrix, save_file)
            matrices.append(matrix)

# MEMORY ERROR
# with open(f"./data/downloaded/matrices.pickle", 'wb') as f:
#     pickle.dump(matrices, f)

100%|██████████| 56/56 [2:01:20<00:00, 130.01s/it]  


In [26]:
# with open(f, "rb") as file:
#     matrices = pickle.load(file)